<img src="media/docker_logo_1.png" width="500" height="500">

It is probably fair to say that access to servers has never been easier.  With platforms such as AWS, Azure, and Google GCE we can now launch on-demand servers of all varieties and configurations.  This programable infrastructure (IaaS) help companies, agencies, and institutions maintain agility as market and mission pressures evolve. However, even with the rise of IaaS, application packaging, configuration, and composition has not advanced despite considerable efforts in configuration management.  This is where [`docker`](https://www.docker.com/) comes in.  

`Docker` is not about full virtualization but rather about the ease of packaging and running applications using [Linux containers](https://en.wikipedia.org/wiki/LXC).  The idea is that `docker` containers wrap a piece of software or application in a complete filesystem that contains everything needed to run: code, runtime, system tools, system libraries (i.e. anything that can be installed on a server). This guarantees that the software will always run the same everywhere, regardless of the OS/compute environment running the container. Docker also provides portable Linux deployment such that containers can be run on any Linux system with kernel is 3.10 or later.  All major Linux distros have supported Docker since 2014.  While no doubt containers and virtual machines have similar resource isolation and allocation benefits, the architectural approach of Linux containers allows containerized applications to be more portable and efficient.

At NVIDIA, we use containers in a variety of ways including development, testing, benchmarking, and of course in production as the mechanism for deploying deep learning frameworks. Using [`nvidia-docker`](https://github.com/NVIDIA/nvidia-docker), a light-weight `docker` plugin, we can develop and prototype GPU applications on a workstation, and then deploy those applications anywhere that supports GPU containers.

# Setup
In the interest of time, we've already configured docker and nvidia-docker. If you're interested in setup details, see Appendix A at the bottom.

# First Contact

The simplest way to interact with `docker` is probably to just ask for the version information

In [11]:
docker --version

Docker version 1.12.3, build 6b644ec


We can ask `nvidia-docker` for the version information too

In [12]:
nvidia-docker --version

Docker version 1.12.3, build 6b644ec


Notice that `nvidia-docker` invocation here was simply "pass through" to `docker` command itself.

Next best way to get familiar with docker command line is to ask for `--help`

In [13]:
docker --help

Usage: docker [OPTIONS] COMMAND [arg...]
       docker [ --help | -v | --version ]

A self-sufficient runtime for containers.

Options:

  --config=~/.docker              Location of client config files
  -D, --debug                     Enable debug mode
  -H, --host=[]                   Daemon socket(s) to connect to
  -h, --help                      Print usage
  -l, --log-level=info            Set the logging level
  --tls                           Use TLS; implied by --tlsverify
  --tlscacert=~/.docker/ca.pem    Trust certs signed only by this CA
  --tlscert=~/.docker/cert.pem    Path to TLS certificate file
  --tlskey=~/.docker/key.pem      Path to TLS key file
  --tlsverify                     Use TLS and verify the remote
  -v, --version                   Print version information and quit

Commands:
    attach    Attach to a running container
    build     Build an image from a Dockerfile
    commit    Create a new image from a container's changes
    cp        Copy files/folde

The format of `docker` command line interactions is: 

`docker [OPTIONS] COMMAND [arg...]` 

and as the help display shows there are a lot of commands to choose from.  Don't worry, much like a big city, once we become more familiar with these commands the list won't seem so big.  We can start to drill down and get help that is specific to each command.  For example, one of the most useful `docker` commands is **`images`** which list all local containers on the host that `docker` knows about 

In [15]:
docker images --help


Usage:	docker images [OPTIONS] [REPOSITORY[:TAG]]

List images

Options:
  -a, --all             Show all images (default hides intermediate images)
      --digests         Show digests
  -f, --filter value    Filter output based on conditions provided (default [])
      --format string   Pretty-print images using a Go template
      --help            Print usage
      --no-trunc        Don't truncate output
  -q, --quiet           Only show numeric IDs


OK, lets now ask `docker` about the what container images are available locally on the host

In [10]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
nvidia/cuda         8.0-cudnn5-devel    31582c303549        5 weeks ago         1.776 GB
nvidia/cuda         latest              367795fb1051        5 weeks ago         1.615 GB
hello-world         latest              c54a2cc56cbb        5 months ago        1.848 kB


Here the output specifies three container images with some general metadata associated with each one.  First you'll notice that the images are quite large on average (~ 2GB) and that each image is associated with a unique ID hash.  When containers are created (i.e. via the **`create`** command) they are created from images.  There is no limit to the number of containers we can create from an images so it is important that `docker` associates UUIDs for each image and container.  Notice the `REPOSITORY` and `TAG` columns here specify more human readable image labels.  The repository loosely coresponds to the image name (i.e. url) and just as in the version control system [GIT](https://git-scm.com/) images can be modified and "tagged" rather than explicitly changing the image name for each image version. 

Here we have the "`nvidia/cuda`" container with ID `c54a2cc56cbb` and is tagged as the "`latest`" version of the image (i.e. most current).  The deep learning library [`cuDNN`](https://developer.nvidia.com/cudnn) was added to the image and a new image was created under the same name but tagged appropriately as "`8.0-cudnn5-devel`".

You're probably wondering already "*where does docker store these containers?*".  In general, docker works in `/var/lib/docker` and images are stored in `image` subdirectory.  For more information and details about where and how docker stores images on the host machine, see [here](https://stackoverflow.com/questions/19234831/where-are-docker-images-stored-on-the-host-machine#). 

For now just know that docker works with "images" and all containers are created from these images.  We will go into all the details about creating and modifying images etc in just a bit.  But first, lets actually kick around some containers!

## Getting Started with Containers

First things first lets have docker list *all* containers using the **`ps`** command

In [19]:
docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


Again, don't forget you can get help for each command with `docker [COMMAND] --help`.  Use this to get additional details on the **`ps`** command.

Lets now use the docker command **`create`** to initialize a container from the `nvidia/cuda:latest` image

In [20]:
docker create nvidia/cuda:latest

9a12d94b12fd4e3de47a3ee3bab4d771d80b6895b575dc2c1ad257cb0f3d121f


The responce we recieved is a sha256 UUID for the generated container and listing `docker` containers again we see this new container now listed 

In [21]:
docker ps -a

CONTAINER ID        IMAGE                COMMAND             CREATED             STATUS              PORTS               NAMES
9a12d94b12fd        nvidia/cuda:latest   "/bin/bash"         45 seconds ago      Created                                 lonely_easley


It is important to understand that the container is not actually doing anything right now.  We've only "stamped" out a container from an image -- the container is not running at this point.  Were the container active the `STATUS` would read "running".

In [3]:
nvidia-docker run --rm nvidia/cuda:8.0-cudnn5-devel nvidia-smi

Sat Nov 12 21:09:56 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   45C    P8    28W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [5]:
docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [6]:
nvidia-docker run nvidia/cuda:8.0-cudnn5-devel nvidia-smi

Sat Nov 12 21:11:30 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   45C    P8    28W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [7]:
docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [8]:
docker ps -a

CONTAINER ID        IMAGE                          COMMAND             CREATED             STATUS                      PORTS               NAMES
e310b48e17f9        nvidia/cuda:8.0-cudnn5-devel   "nvidia-smi"        12 seconds ago      Exited (0) 11 seconds ago                       backstabbing_engelbart


In [9]:
nvidia-docker start e310b48e17f9

e310b48e17f9


In [10]:
nvidia-docker start --help


Usage:	docker start [OPTIONS] CONTAINER [CONTAINER...]

Start one or more stopped containers

Options:
  -a, --attach               Attach STDOUT/STDERR and forward signals
      --detach-keys string   Override the key sequence for detaching a container
      --help                 Print usage
  -i, --interactive          Attach container's STDIN


In [11]:
nvidia-docker start -i e310b48e17f9

Sat Nov 12 21:16:36 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   45C    P8    28W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [12]:
nvidia-docker start -a e310b48e17f9

Sat Nov 12 21:17:01 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 0000:00:1E.0     Off |                    0 |
| N/A   45C    P8    28W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [13]:
nvidia-docker exec e310b48e17f9 echo "hello from nvidia"

Error response from daemon: Container e310b48e17f97c1103c53a3c91d008581019782cb523462c68dbbcebc4043e54 is not running


In [14]:
nvidia-docker run --interactive --tty -d e310b48e17f9 /bin/bash

Using default tag: latest
Pulling repository docker.io/library/e310b48e17f9
nvidia-docker | 2016/11/12 21:29:35 Error: image library/e310b48e17f9:latest not found


In [16]:
docker ps -a

CONTAINER ID        IMAGE                          COMMAND             CREATED             STATUS                      PORTS               NAMES
e310b48e17f9        nvidia/cuda:8.0-cudnn5-devel   "nvidia-smi"        20 minutes ago      Exited (0) 14 minutes ago                       backstabbing_engelbart


In [17]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
nvidia/cuda         8.0-cudnn5-devel    31582c303549        3 weeks ago         1.776 GB
nvidia/cuda         latest              367795fb1051        3 weeks ago         1.615 GB
hello-world         latest              c54a2cc56cbb        4 months ago        1.848 kB


In [18]:
nvidia-docker run --interactive --tty -d nvidia/cuda:8.0-cudnn5-devel /bin/bash

1868fbf74e0b9e613b4a41760f2b31257c280624a3b70d4705d4cb8c4f519150


In [19]:
docker ps 

CONTAINER ID        IMAGE                          COMMAND             CREATED             STATUS              PORTS               NAMES
1868fbf74e0b        nvidia/cuda:8.0-cudnn5-devel   "/bin/bash"         16 seconds ago      Up 15 seconds                           stupefied_bhabha


In [26]:
nvidia-docker exec 1868fbf74e0b ls /usr/local/cuda/extras

CUPTI
Debugger


# Appendix A
Here are some installation details for getting docker up and running from scratch ...

### Step 0 - GPU Driver
In order to get GPU access with in docker/nvidia-docker we need to make sure that the NVIDIA driver is available on the host system.  It's possible to obtain the appropriate device driver from either the standard [driver download](http://www.nvidia.com/download/index.aspx) page or via [CUDA installation](https://developer.nvidia.com/cuda-downloads).

### Step 1 - Docker Install
Once the NVIDIA device driver has been successfully installed, we need to install `docker` it self. The installation of docker is quite simple but it is just slightly different for each OS.  The steps for `docker` installation on Ubuntu 14.04 can be found [here](https://docs.docker.com/engine/installation/linux/ubuntulinux/).  Don't worry, the [`docker` docs](https://docs.docker.com/) have install instructions for many other operating systems including RedHat, CentOS, Debian, and so on.

Docker provides an official installation script via https://get.docker.com which can accessed via command-line using "`wget -qO-`" or "`curl -sSL`"

### Step 2 - NVIDIA Docker 
The final configuration step is to obtain the `nvidia-docker` plugin which properly exposes the GPU hardware and drivers for `docker` containers.  Official installation instructions for `nvidia-docker` for Ubuntu, CentOS, and other distributions can be found [here](https://github.com/NVIDIA/nvidia-docker)